In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}
pd.set_option('display.float_format','{:,.2f}'.format)
year = 2023
refund = 102_000   # 112,167.31
dividend = 325_000
expense  = 510_000

today = date.today()
today

datetime.date(2023, 7, 15)

### Restart and Run All Cells

### Record selection for transactions

In [2]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % year
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2023
ORDER BY sells.date DESC, name


In [3]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,net,kind
0,KTC,2023,7,2023,6,45.00,50.00,-5.00,900,"40,500.00","45,000.00","-4,500.00",-10.00,"-4,689.38",DTD
1,JMART,2023,7,2023,6,17.90,17.00,0.90,"1,800","32,220.00","30,600.00","1,620.00",5.29,"1,480.86",DTD
2,BANPU,2023,6,2023,3,8.70,10.80,-2.10,"6,000","52,200.00","64,800.00","-12,600.00",-19.44,"-12,859.14",HD
3,BANPU,2023,6,2023,1,8.70,11.40,-2.70,"6,000","52,200.00","68,400.00","-16,200.00",-23.68,"-16,467.11",HD
4,BANPU,2023,6,2023,1,8.70,12.30,-3.60,"12,000","104,400.00","147,600.00","-43,200.00",-29.27,"-43,758.16",DTD
5,BANPU,2023,6,2023,3,8.70,10.90,-2.20,"6,000","52,200.00","65,400.00","-13,200.00",-20.18,"-13,460.47",HD
6,DIF,2023,6,2023,6,11.20,10.60,0.60,"2,000","22,400.00","21,200.00","1,200.00",5.66,"1,103.43",DTD
7,EA,2023,6,2023,6,55.75,53.00,2.75,400,"22,300.00","21,200.00","1,100.00",5.19,"1,003.66",DTD
8,CPNREIT,2023,6,2023,6,10.70,10.20,0.50,"3,000","32,100.00","30,600.00","1,500.00",4.90,"1,361.13",HD
9,TFFIF,2023,6,2023,5,7.70,7.65,0.05,"10,000","77,000.00","76,500.00",500.00,0.65,160.00,HD


In [4]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum()

sell_year  sell_month  buy_year  buy_month  name   
2023       1           2021      9          STA       -63,619.06
                                 10         STA       -29,173.73
                       2022      2          CPAXT      14,372.89
                                            PTTGC     -11,197.69
                                 6          KCE        -7,754.72
                                 12         GFPT        4,074.74
                                            JMART       5,455.13
                       2023      1          SINGER      5,847.49
                                            WHAIR         656.69
           2           2021      7          STA       -34,467.49
                       2022      2          SCCC      -15,875.76
                                 3          KCE       -13,762.21
                                 4          KCE        -7,013.01
                                 5          DCC        -6,773.43
           3           2020      10   

In [5]:
sells_df.groupby(['name'])[['gross','net']].sum().head(5)

,gross,net
name,,
BANPU,"-85,200.00","-86,544.88"
CPAXT,"15,750.00","14,372.89"
CPNREIT,"1,500.00","1,361.13"
DCC,"-6,000.00","-6,773.43"
DIF,"-12,800.00","-13,516.74"


In [6]:
total_df = sells_df.groupby(['name'])[['gross','net']].sum()
total_df.nlargest(5, 'gross')[['gross','net']].style.format(format_dict)

,gross,net
name,,
CPAXT,"15,750.00","14,372.89"
JMART,"7,620.00","6,935.99"
SINGER,"6,300.00","5,847.49"
JMT,"5,250.00","4,922.09"
GVREIT,"5,000.00","4,594.67"


In [7]:
file_name = 'ytd-profit.csv'
data_file = data_path + file_name
csv_file = csv_path + file_name
osd_file = osd_path + file_name

sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(csv_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(data_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(osd_file)

In [8]:
output = sells_df.groupby(['sell_month']).net.sum()
output

sell_month
1   -81,338.26
2   -77,891.90
3    -7,863.28
5     3,680.06
6   -75,562.92
7    -3,208.52
Name: net, dtype: float64

In [9]:
data_path = "../data/"
file_name = 'tmp-file-of-profits.csv'
data_file = data_path + file_name
osd_file = osd_path + file_name

output.to_csv(data_file, index=True)
output.to_csv(osd_file, index=True)

In [10]:
profit = sells_df.net.sum()
ttl_profit = profit + dividend
profit,dividend,ttl_profit

(-242184.82, 325000, 82815.18)

In [11]:
profit = sells_df.net.sum()
net = profit + dividend - expense
round(profit,0),dividend,expense, round(net,0)

(-242185.0, 325000, 510000, -427185.0)

### Bottom line of up to date transactions

In [12]:
trade = round(profit,0) + dividend
trade

82815.0

In [13]:
total = refund + net
total

-325184.82